[simpleT5](https://github.com/Shivanandroy/simpleT5)

mT5(multilingual T5)의 ROUGE-N에 대한 F1 SCORE 0.745570.676280.72491

In [ ]:
!pip install simplet5

In [ ]:
import json
import numpy as np
import pandas as pd

In [ ]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [ ]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [ ]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [ ]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [ ]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = train.rename(columns={"summary":"target_text", "total":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="mt5", model_name="google/mt5-small")
model.train(train_df=train_df,
            eval_df=test_df, 
            # source_max_token_len=1724, 
            target_max_token_len=139, 
            batch_size=8, max_epochs=100, early_stopping_patience_epochs = 2, use_gpu=True)

In [ ]:
text_to_summarize = test['total'].tolist()

In [ ]:
# let's load the trained model for inferencing:

result = []

model.load_model("mt5","/content/outputs/simplet5-epoch-29-train-loss-0.7763", use_gpu=True)

for i in range(len(text_to_summarize)):
  result.append(model.predict(text_to_summarize[i]))

result[:10]

In [ ]:
final_result = sum(result, [])

In [ ]:
final_result[:10]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_LG/data/sample_submission.csv')

In [ ]:
submission['summary'] = final_result

In [ ]:
submission.head()

In [ ]:
submission.to_csv('/content/drive/MyDrive/DACON_LG/data/mt5-trial.csv', index=False)